In [94]:
import pandas as pd
data=pd.read_excel(r"C:\Users\SAMSUNG\Desktop\추가데이터\전체자료_관공서인프라신사동수정+평균고도+하천까지의거리추가.xlsx")
data["침수여부"]=0
data.loc[data["침수횟수"]>=1,"침수여부"]=1
data["침수여부"].value_counts()
data['구단위시우량']=data['동단위우량']/data["동면적(k㎡)"]

In [95]:
data

,Unnamed: 0,GU_NAME,ADM_NM,동면적(k㎡),인구수,인구밀도(명/km²),동단위우량,동단위시우량,침수횟수,경사도,...,고령자비율,장애인비율,행정동코드,면적,지하건물,빗물펌프장까지의거리,평균고도,하천까지의거리,침수여부,구단위시우량
0,0,종로구,사직동,1.23,9371,7618.699187,9.062665,0.379032,0,0.153750,...,0.193266,0.028541,1101053,1.23,558,4389.596516,44.422401,0.909096,0,7.36802
1,1,종로구,삼청동,1.49,2616,1755.704698,10.978350,0.459153,0,0.250000,...,0.236942,0.041635,1101054,1.49,455,4633.405055,107.956271,0.490531,0,7.36802
2,2,종로구,부암동,2.27,9460,4167.400881,16.725406,0.699515,2,0.314583,...,0.190227,0.032613,1101055,2.27,813,5699.673727,147.729398,0.579168,1,7.36802
3,3,종로구,평창동,8.87,17743,2000.338219,65.354340,2.733348,2,0.229615,...,0.195711,0.027495,1101056,8.87,1797,5465.810242,273.213812,0.104944,1,7.36802
4,4,종로구,무악동,0.36,8045,22347.222220,2.652487,0.110936,0,0.346667,...,0.183308,0.040363,1101057,0.36,58,4182.759161,113.928953,1.427201,0,7.36802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,402,강동구,성내3동,0.71,22984,32371.830990,6.245160,0.253972,12,0.072143,...,0.177626,0.000130,1125067,0.71,788,1179.328824,17.434927,1.346759,1,8.79600
403,403,강동구,길동,1.61,45225,28090.062110,14.161560,0.575907,9,0.176154,...,0.186847,0.036607,1125074,1.61,1319,1960.757507,28.949684,1.073927,1,8.79600
404,404,강동구,둔촌1동,0.92,74,80.434783,8.092320,0.329090,3,0.151786,...,0.150685,12.671233,1125070,0.92,6,1646.603436,30.338302,1.163773,1,8.79600
405,405,강동구,둔촌2동,1.56,24212,15520.512820,13.721760,0.558022,10,0.170909,...,0.175063,0.069696,1125071,1.56,714,2376.349662,39.652645,0.369031,1,8.79600


In [96]:
target="침수여부"
use_feature=["구단위시우량","경사도","양호비율","도로_건물_면적_비율","녹지_면적_비율","제방","수문","암거","노후건물개수","빗물펌프장까지의거리","지하건물","평균고도",'하천까지의거리']

In [97]:
data[use_feature]

,구단위시우량,경사도,양호비율,도로_건물_면적_비율,녹지_면적_비율,제방,수문,암거,노후건물개수,빗물펌프장까지의거리,지하건물,평균고도,하천까지의거리
0,7.36802,0.153750,0.245283,0.307107,0.386556,3,0,95,485,4389.596516,558,44.422401,0.909096
1,7.36802,0.250000,0.832402,0.137073,0.641702,0,0,43,344,4633.405055,455,107.956271,0.490531
2,7.36802,0.314583,0.906250,0.147000,0.225413,40,3,40,972,5699.673727,813,147.729398,0.579168
3,7.36802,0.229615,0.974132,0.087765,0.164587,83,20,171,1331,5465.810242,1797,273.213812,0.104944
4,7.36802,0.346667,1.000000,0.169418,0.461577,0,0,10,101,4182.759161,58,113.928953,1.427201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,8.79600,0.072143,0.633929,0.438031,0.005765,0,0,17,425,1179.328824,788,17.434927,1.346759
403,8.79600,0.176154,0.608911,0.358275,0.059004,4,0,21,892,1960.757507,1319,28.949684,1.073927
404,8.79600,0.151786,0.738636,0.229434,0.045133,0,0,9,11,1646.603436,6,30.338302,1.163773
405,8.79600,0.170909,0.797872,0.224797,0.147004,19,1,68,574,2376.349662,714,39.652645,0.369031


In [98]:
# 1, Random_Forest
import sklearn.ensemble as skens
import sklearn.model_selection as skmod
import sklearn.preprocessing as skpre
import numpy as np
import scipy as sp
np.random.seed(10)

#data,test=skmod.data_test_split(data[[target]+use_feature],data_size=0.7,random_state=10,stratify=data[target])
RT_model=skens.RandomForestClassifier(n_estimators=200,criterion='entropy',random_state=10,n_jobs=-1,oob_score=True)
RT_model.fit(X=data[use_feature],y=data[target])

RandomForestClassifier(criterion='entropy', n_estimators=200, n_jobs=-1,
                       oob_score=True, random_state=10)

In [99]:
#Random Forest 모델성능
RT_model.oob_score_

0.7321867321867321

In [100]:
#SVC


In [114]:
#KNN
import sklearn.neighbors as sknei
import sklearn.ensemble as skens
import sklearn.metrics as skmet

def cosine_distance(X, Y=None):
    return skmet.pairwise_distances(X, Y, metric='cosine')

inner_model=sknei.KNeighborsClassifier(n_neighbors=k,n_jobs=-1) # 우선 k=5 기본값 사용

 #랜덤포레스트랑 동일하게
knn_ensemble=skens.BaggingClassifier(estimator=inner_model,n_estimators=100,n_jobs=-1,random_state=10,oob_score=True,bootstrap_features=True)

In [115]:
import sklearn.preprocessing as skpre
minmax=skpre.MinMaxScaler()
scale_data=data.copy()
scale_data[use_feature]=minmax.fit_transform(X=scale_data[use_feature])

In [126]:
# k값찾기
import scipy as sp
k=int(np.sqrt(len(data)*2/3)) # 407개 데이터 중 2/3정도가 부트스트래핑으로 사용되므로 sqrt(data_set_size)와 가까운 홀수 선정
params={"n_neighbors":sp.stats.randint(5,k)}
random_search = skmod.RandomizedSearchCV(knn_ensemble,
 param_distributions=params, n_iter=10, cv=5, scoring='accuracy', random_state=10)

In [133]:
random_search.fit(X=scale_data[use_feature],y=scale_data[target])

ValueError: Invalid parameter 'n_neighbors' for estimator BaggingClassifier(estimator=KNeighborsClassifier(n_jobs=-1, n_neighbors=15),
                  n_estimators=100, n_jobs=-1, oob_score=True, random_state=10). Valid parameters are: ['base_estimator', 'bootstrap', 'bootstrap_features', 'estimator', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].

In [125]:
best_k_search=skmod.RandomizedSearchCV

In [116]:
knn_ensemble.fit(X=scale_data[use_feature],y=scale_data[target])

BaggingClassifier(estimator=KNeighborsClassifier(n_jobs=-1, n_neighbors=15),
                  n_estimators=100, n_jobs=-1, oob_score=True, random_state=10)

In [117]:
knn_ensemble.oob_score_

0.6904176904176904

In [106]:
import sklearn 

In [107]:
print(dir(sklearn))

['__SKLEARN_SETUP__', '__all__', '__builtins__', '__cached__', '__check_build', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_config', '_distributor_init', '_loss', 'base', 'clone', 'config_context', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'externals', 'get_config', 'linear_model', 'logger', 'logging', 'metrics', 'model_selection', 'neighbors', 'os', 'preprocessing', 'random', 'set_config', 'setup_module', 'show_versions', 'svm', 'sys', 'tree', 'utils']
